# Exercise 4: Training a Model (GPU)

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
x = torch.rand((2,3,4))
x.device

device(type='cpu')

In [ ]:
x = x.to(device)
x.device

device(type='cuda', index=0)

# Data Loading and Preprocessing

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = datasets.CIFAR10(
                root='./data', 
                train=True,
                download=True, 
                transform=transform_train)
trainloader = DataLoader(
                trainset, 
                batch_size=32,
                shuffle=True)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Extracting ./data/cifar-10-python.tar.gz to ./data


# Model Design

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = LeNet()

# Define Loss Function and Optimizer

In [ ]:
import torch.optim as optim

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), 
                      lr=0.001, 
                      momentum=0.9)

In [ ]:
model.to(device)                                #### GPU

for epoch in range(5):
  running_loss = 0
  for inputs, labels in trainloader:
    optimizer.zero_grad()
    outputs = model(inputs.to(device))          #### GPU
    loss = loss_fn(outputs, labels.to(device))  #### GPU
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  
  print(f"Average Loss per Epoch:", running_loss/len(trainloader))

Average Loss per Epoch: 2.081776667236138
Average Loss per Epoch: 1.7247091567600223
Average Loss per Epoch: 1.5949005734363735
Average Loss per Epoch: 1.5164100259294588
Average Loss per Epoch: 1.4575592195537712


# Test Model Performance

In [ ]:
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

testset = datasets.CIFAR10(
                root='./data', 
                train=False,
                download=True, 
                transform=transform_test)
testloader = DataLoader(
                testset, 
                batch_size=32,
                shuffle=True)

Files already downloaded and verified


In [ ]:
correct = 0
with torch.no_grad():
    for inputs, labels in testloader:
        outputs = model(inputs.to(device))          #### GPU
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()

print("Model accuracy on %d Test Images: %.2f %%" % 
        (len(testset), (correct/len(testset) * 100)))

Model accuracy on 10000 Test Images: 52.15 %
